In [55]:
import json
from typing import (Any, Dict, List, Literal, Optional, Sequence, TypedDict,
                    Union)

import nest_asyncio
from langgraph.graph import END, StateGraph
from llama_index.core.base.llms.types import (ChatMessage, ChatResponse,
                                              MessageRole)
from llama_index.core.llms import ChatMessage
from llama_index.core.prompts.base import ChatPromptTemplate
from llama_index.core.tools.function_tool import FunctionTool
from llama_index.core.tools.types import BaseTool, ToolOutput
from llama_index.llms.groq import Groq
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from loguru import logger
from openai.types.chat import ChatCompletionMessageToolCall
from rich.pretty import pprint

nest_asyncio.apply()

def pretty_print(title: str = None, content: Any = None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)


## Tools

In [56]:
def int_mult(a: int, b: int) -> int:
    """Apply a * b and returns the result as int"""
    res = a * b
    logger.debug(f"Apply {a} * {b} and returns {res}")

    return res

def int_add(a: int, b: int) -> int:
    """Apply a + b and returns the result as int"""
    res = a + b
    logger.debug(f"Apply {a} + {b} and returns {res}")

    return res


def random_joke()->str:
    """A crazy creator that can create a random joke."""
    #model: OpenAI = OpenAI(temperature=1.5)
    model = Ollama(model="gemma:2b-instruct", temperature=2.)
    res = model.complete(f"You are a crazy creator that can create a random joke. Notice: only return the joke content without any other information, empty spaces, leading text, newlines or instruction information.")
    return res.text.strip()


def translate(string: str)->str:
    """Translate the string into Simplified Chinese."""
    #model: OpenAI = OpenAI(temperature=0)
    model = Ollama(model="gemma:2b-instruct", temperature=0.5)
    res = model.complete(f"Translate the text inside [TEXT] into Simplified Chinese, only return the result. [TEXT]{string}[TEXT]")
    return res.text.strip()

# Some demos:
a_joke = random_joke()
joke_translated = translate(a_joke)
pprint(joke_translated)

"What do you call a stick that won't come back?\n\n答案：一根无法回家的棍子。"

### Convert Python functiont to function-tool.

Similar to `@tool` in LangChain

In [57]:
int_mult_tool, int_add_tool = FunctionTool.from_defaults(fn=int_mult), FunctionTool.from_defaults(fn=int_add)
int_tools: Sequence[BaseTool]  = [int_mult_tool, int_add_tool]

pprint(int_tools)

[
│   <llama_index.core.tools.function_tool.FunctionTool object at 0x134c75d50>,
│   <llama_index.core.tools.function_tool.FunctionTool object at 0x13427b790>
]

In [58]:
for tool in int_tools:
    pprint(tool.metadata)
    print("-----")

ToolMetadata(
│   description='int_mult(a: int, b: int) -> int\nApply a * b and returns the result as int',
│   name='int_mult',
│   fn_schema=<class 'pydantic.main.int_mult'>
)

-----


ToolMetadata(
│   description='int_add(a: int, b: int) -> int\nApply a + b and returns the result as int',
│   name='int_add',
│   fn_schema=<class 'pydantic.main.int_add'>
)

-----


In [59]:
tools_dict: Dict[str, FunctionTool] = {tool.metadata.name: tool for tool in int_tools}
pprint(tools_dict)

{
│   'int_mult': <llama_index.core.tools.function_tool.FunctionTool object at 0x134c75d50>,
│   'int_add': <llama_index.core.tools.function_tool.FunctionTool object at 0x13427b790>
}

### Call a tool

In [60]:
int_mult_tool(**{"a":12,"b":34})

2024-03-05 21:18:00.209 | DEBUG    | __main__:int_mult:4 - Apply 12 * 34 and returns 408


ToolOutput(content='408', tool_name='int_mult', raw_input={'args': (), 'kwargs': {'a': 12, 'b': 34}}, raw_output=408)

## Model

In [61]:
model: OpenAI = OpenAI(temperature=0, model="gpt-4-1106-preview")

## Prompt template

Ref: https://docs.llamaindex.ai/en/stable/examples/customization/prompts/chat_prompts.html



In [62]:
messages: Sequence[ChatMessage] = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=("You are an assisant to perform the user input."),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=("{input}"),
    ),
]
tmpl: ChatPromptTemplate = ChatPromptTemplate(messages)

logger.debug("in string")

messages_str: str = tmpl.format(input="hi, template")
pprint(messages_str)

logger.debug("in list")

messages: Sequence[ChatMessage] = tmpl.format_messages(input="hi, messages")
pprint(messages)

2024-03-05 21:18:00.308 | DEBUG    | __main__:<module>:13 - in string


'system: You are an assisant to perform the user input.\nuser: hi, template\nassistant: '

2024-03-05 21:18:00.314 | DEBUG    | __main__:<module>:18 - in list


[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(role=<MessageRole.USER: 'user'>, content='hi, messages', additional_kwargs={})
]

## Query with tools

In [63]:
messages: Sequence[ChatMessage]  = tmpl.format_messages(input="What is the result of the 12*34?")
pprint(messages)

[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.USER: 'user'>,
│   │   content='What is the result of the 12*34?',
│   │   additional_kwargs={}
│   )
]

### Notice

Cannot call `model.chat(messages, tools=int_tools)`, it complaints, because it is not OpenAI format.

```
TypeError                                 Traceback (most recent call last)
<ipython-input-26-76230e6dd870> in <cell line: 1>()
----> 1 res = model.chat(messages, tools=int_tools)
      2 pprint(res)

16 frames
/usr/lib/python3.10/json/encoder.py in default(self, o)
    177
    178         """
--> 179         raise TypeError(f'Object of type {o.__class__.__name__} '
    180                         f'is not JSON serializable')
    181

TypeError: Object of type FunctionTool is not JSON serializable
```



### Convert to OpenAI function

Similar to LangChain `langchain_core.utils.function_calling.convert_to_openai_function`

In [64]:
openai_tools: Dict[str, Any] = [
            tool.metadata.to_openai_tool() for tool in int_tools
        ]
pprint(openai_tools)

[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'int_mult',
│   │   │   'description': 'int_mult(a: int, b: int) -> int\nApply a * b and returns the result as int',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}},
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'int_add',
│   │   │   'description': 'int_add(a: int, b: int) -> int\nApply a + b and returns the result as int',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}},
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   }
]

### Useful query

In [65]:
res:ChatResponse = model.chat(messages, tools=openai_tools)
pprint(res)

ChatResponse(
│   message=ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content=None,
│   │   additional_kwargs={
│   │   │   'tool_calls': [
│   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   id='call_uFap5QFPTzJXIW7Yemnt4aAT',
│   │   │   │   │   function=Function(arguments='{"a":12,"b":34}', name='int_mult'),
│   │   │   │   │   type='function'
│   │   │   │   )
│   │   │   ]
│   │   }
│   ),
│   raw={
│   │   'id': 'chatcmpl-8zVWadfa5hALPaMyMyIHsHUKTMB0S',
│   │   'choices': [
│   │   │   Choice(
│   │   │   │   finish_reason='tool_calls',
│   │   │   │   index=0,
│   │   │   │   logprobs=None,
│   │   │   │   message=ChatCompletionMessage(
│   │   │   │   │   content=None,
│   │   │   │   │   role='assistant',
│   │   │   │   │   function_call=None,
│   │   │   │   │   tool_calls=[
│   │   │   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   │   │   id='call_uFap5QFPTzJXIW7Yemnt4aAT',
│   │   │   │   │   │   │   function=Function(arguments='{"a":12,"b":34}', name='int_mult'),
│   │   │   │   │   │   │   type='function'
│   │   │   │   │   │   )
│   │   │   │   │   ]
│   │   │   │   )
│   │   │   )
│   │   ],
│   │   'created': 1709669880,
│   │   'model': 'gpt-4-1106-preview',
│   │   'object': 'chat.completion',
│   │   'system_fingerprint': 'fp_021f187978',
│   │   'usage': CompletionUsage(completion_tokens=18, prompt_tokens=148, total_tokens=166)
│   },
│   delta=None,
│   additional_kwargs={}
)

### Useless query, no tool required

In [66]:
messages: Sequence[ChatMessage]  = tmpl.format_messages(input="hi")
pprint(messages)

[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(role=<MessageRole.USER: 'user'>, content='hi', additional_kwargs={})
]

In [67]:
res: ChatResponse = model.chat(messages, tools=openai_tools)
pprint(res)

ChatResponse(
│   message=ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content='Hello! How can I assist you today?',
│   │   additional_kwargs={}
│   ),
│   raw={
│   │   'id': 'chatcmpl-8zVWdBMFXs1jhaKSBYeIUmavAXfCs',
│   │   'choices': [
│   │   │   Choice(
│   │   │   │   finish_reason='stop',
│   │   │   │   index=0,
│   │   │   │   logprobs=None,
│   │   │   │   message=ChatCompletionMessage(
│   │   │   │   │   content='Hello! How can I assist you today?',
│   │   │   │   │   role='assistant',
│   │   │   │   │   function_call=None,
│   │   │   │   │   tool_calls=None
│   │   │   │   )
│   │   │   )
│   │   ],
│   │   'created': 1709669883,
│   │   'model': 'gpt-4-1106-preview',
│   │   'object': 'chat.completion',
│   │   'system_fingerprint': 'fp_021f187978',
│   │   'usage': CompletionUsage(completion_tokens=10, prompt_tokens=138, total_tokens=148)
│   },
│   delta=None,
│   additional_kwargs={}
)

## Agent loop

In [115]:
class AgentExecutor:
    def __init__(self, tools: Sequence[Any]):
        func_tools: Sequence[BaseTool] = list(
            map(lambda f: FunctionTool.from_defaults(fn=f), tools)
        )
        self.func_tools_dict: Dict[str, FunctionTool] = {
            tool.metadata.name: tool for tool in func_tools
        }
        self.openai_tools: Sequence[Dict[str, Any]] = list(
            map(lambda func_tool: func_tool.metadata.to_openai_tool(), func_tools)
        )

        self.chat_history: List[ChatMessage] = list()

        messages: Sequence[ChatMessage] = [
            ChatMessage(
                role=MessageRole.SYSTEM,
                content=("You are an assisant to perform the user input."),
            ),
            ChatMessage(
                role=MessageRole.USER,
                content=("{input}"),
            ),
        ]
        self.chat_prompt_template: ChatPromptTemplate = ChatPromptTemplate(messages)
        self.model: OpenAI = OpenAI(temperature=0, model="gpt-4-1106-preview")
        # self.model: Groq = Groq(
        #     model="mixtral-8x7b-32768",
        #     temperature=0,
        #     timeout=60,
        # )

    def should_continue(self, chat_response: ChatResponse) -> bool:
        return (
            chat_response.message.additional_kwargs.get("tool_calls", None) is not None
        )

    def run_tool(self, tool_call: ChatCompletionMessageToolCall) -> ChatMessage:
        func_id: str = tool_call.id
        func_name: str = tool_call.function.name
        args_json: str = tool_call.function.arguments

        func: FunctionTool = self.func_tools_dict.get(func_name)
        res: ToolOutput = func(**json.loads(args_json))
        pretty_print("Ran tool", res)
        return ChatMessage(
            role=MessageRole.TOOL,
            content=str(res),
            name=func_name,
            additional_kwargs={
                "tool_call_id": func_id,
                "name": func_name,
            },
        )

    def __call__(self, human_input: str) -> ChatMessage:
        pretty_print("Agent on start.")
        messages: Sequence[ChatMessage] = self.chat_prompt_template.format_messages(
            input=human_input
        )
        self.chat_history.extend(messages)
        while True:
            chat_response: ChatResponse = self.model.chat(
                messages, tools=self.openai_tools
            )
            ai_message: ChatMessage = chat_response.message
            pretty_print("AI messsaging", ai_message)

            self.chat_history.extend([ai_message])
            if not self.should_continue(chat_response):
                pretty_print("Agent on stop.")
                return ai_message
            else:
                func_message = self.run_tool(
                    ai_message.additional_kwargs["tool_calls"][0]
                )
                self.chat_history.extend([func_message])
                messages = self.chat_history
                logger.info("Agent in continue.")

    def __str__(self) -> str:
        pretty_print("func_tools_dict", self.func_tools_dict)
        pretty_print("openai_tools", self.openai_tools)
        pretty_print("chat_prompt_template", self.chat_prompt_template)
        pretty_print("chat_history", self.chat_history)


agent_exe: AgentExecutor = AgentExecutor(tools=[random_joke, translate])

### Run agent

In [116]:
#@title { vertical-output: true}

human_input: str = """
Please create a random joke and provide the translated result in Simplified Chinese
without including the original language, newlines, empty spaces, or instruction information
"""

res: ChatMessage = agent_exe(human_input=human_input)
print("\n---Final result---")
pprint(res)

2024-03-05 21:47:31.513 | INFO     | __main__:__call__:57 - Agent on start.


AI messsaging


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content=None,
│   additional_kwargs={
│   │   'tool_calls': [
│   │   │   ChatCompletionMessageToolCall(
│   │   │   │   id='call_lL6XOicfBYiacC9bghzqo8AW',
│   │   │   │   function=Function(arguments='{}', name='random_joke'),
│   │   │   │   type='function'
│   │   │   )
│   │   ]
│   }
)

Ran tool


ToolOutput(
│   content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!',
│   tool_name='random_joke',
│   raw_input={'args': (), 'kwargs': {}},
│   raw_output='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'
)

...

AI messsaging


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content='因为他出色地在这领域获得了奖项！',
│   additional_kwargs={}
)

2024-03-05 21:47:48.732 | INFO     | __main__:__call__:71 - Agent on stop.



---Final result---


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content='因为他出色地在这领域获得了奖项！',
│   additional_kwargs={}
)

### Agent history

In [117]:
agent_exe.__str__()

func_tools_dict


{
│   'random_joke': <llama_index.core.tools.function_tool.FunctionTool object at 0x136ae0d90>,
│   'translate': <llama_index.core.tools.function_tool.FunctionTool object at 0x1369b4c50>
}

openai_tools


[
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'random_joke',
│   │   │   'description': 'random_joke() -> str\nA crazy creator that can create a random joke.',
│   │   │   'parameters': {'type': 'object', 'properties': {}}
│   │   }
│   },
│   {
│   │   'type': 'function',
│   │   'function': {
│   │   │   'name': 'translate',
│   │   │   'description': 'translate(string: str) -> str\nTranslate the string into Simplified Chinese.',
│   │   │   'parameters': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {'string': {'title': 'String', 'type': 'string'}},
│   │   │   │   'required': ['string']
│   │   │   }
│   │   }
│   }
]

chat_prompt_template


ChatPromptTemplate(
│   metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>},
│   template_vars=['input'],
│   kwargs={},
│   output_parser=None,
│   template_var_mappings=None,
│   function_mappings=None,
│   message_templates=[
│   │   ChatMessage(
│   │   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   │   content='You are an assisant to perform the user input.',
│   │   │   additional_kwargs={}
│   │   ),
│   │   ChatMessage(role=<MessageRole.USER: 'user'>, content='{input}', additional_kwargs={})
│   ]
)

chat_history


[
│   ChatMessage(
│   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   content='You are an assisant to perform the user input.',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.USER: 'user'>,
│   │   content='\nPlease create a random joke and provide the translated result in Simplified Chinese\nwithout including the original language, newlines, empty spaces, or instruction information\n',
│   │   additional_kwargs={}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content=None,
│   │   additional_kwargs={
│   │   │   'tool_calls': [
│   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   id='call_lL6XOicfBYiacC9bghzqo8AW',
│   │   │   │   │   function=Function(arguments='{}', name='random_joke'),
│   │   │   │   │   type='function'
│   │   │   │   )
│   │   │   ]
│   │   }
│   ),
│   ChatMessage(
│   │   role=<MessageRole.TOOL: 'tool'>,
│   │   content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!',
│   │   additional_kwargs={'tool_call_id': 'call_lL6XOicfBYiacC9bghzqo8AW', 'name': 'random_joke'}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content=None,
│   │   additional_kwargs={
│   │   │   'tool_calls': [
│   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   id='call_DLLibGwgWpaDjUXZRMCgxMhN',
│   │   │   │   │   function=Function(
│   │   │   │   │   │   arguments='{"string":"Why did the scarecrow win an award? Because he was outstanding in his field!"}',
│   │   │   │   │   │   name='translate'
│   │   │   │   │   ),
│   │   │   │   │   type='function'
│   │   │   │   )
│   │   │   ]
│   │   }
│   ),
│   ChatMessage(
│   │   role=<MessageRole.TOOL: 'tool'>,
│   │   content='因为他出色地在这领域获得了奖项！',
│   │   additional_kwargs={'tool_call_id': 'call_DLLibGwgWpaDjUXZRMCgxMhN', 'name': 'translate'}
│   ),
│   ChatMessage(
│   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   content='因为他出色地在这领域获得了奖项！',
│   │   additional_kwargs={}
│   )
]

## OpenAIAgent

This is Llama-Index buildin, it encapsulates a lot details of Agent running and very robust.

In [118]:
from llama_index.agent.openai import OpenAIAgent

func_tools: Sequence[BaseTool]  = list(map(lambda f: FunctionTool.from_defaults(fn=f), [random_joke, translate] ))
openai_agent = OpenAIAgent.from_tools(
    tools=func_tools,
    llm=model,
    verbose=True
)
res = openai_agent.chat(human_input)
print("\nFinal result")
pprint(res)

Added user message to memory: 
Please create a random joke and provide the translated result in Simplified Chinese
without including the original language, newlines, empty spaces, or instruction information

=== Calling Function ===
Calling function: random_joke with args: {}
Got output: Sure, here's the joke you requested:

Why did the scarecrow win an award?

Because he was outstanding in his field!

=== Calling Function ===
Calling function: translate with args: {"string":"Why did the scarecrow win an award? Because he was outstanding in his field!"}
Got output: 因为他出色地工作，他获得了奖项！


Final result


AgentChatResponse(
│   response='因为他出色地工作，他获得了奖项！',
│   sources=[
│   │   ToolOutput(
│   │   │   content="Sure, here's the joke you requested:\n\nWhy did the scarecrow win an award?\n\nBecause he was outstanding in his field!",
│   │   │   tool_name='random_joke',
│   │   │   raw_input={'args': (), 'kwargs': {}},
│   │   │   raw_output="Sure, here's the joke you requested:\n\nWhy did the scarecrow win an award?\n\nBecause he was outstanding in his field!"
│   │   ),
│   │   ToolOutput(
│   │   │   content='因为他出色地工作，他获得了奖项！',
│   │   │   tool_name='translate',
│   │   │   raw_input={
│   │   │   │   'args': (),
│   │   │   │   'kwargs': {
│   │   │   │   │   'string': 'Why did the scarecrow win an award? Because he was outstanding in his field!'
│   │   │   │   }
│   │   │   },
│   │   │   raw_output='因为他出色地工作，他获得了奖项！'
│   │   )
│   ],
│   source_nodes=[]
)

## LangGraph driven

In [124]:
class GraphState(TypedDict):
    messages: Optional[Sequence[ChatMessage]] = (
        None  # the history of the interaction with model
    )
    chat_response: Optional[ChatResponse] = None  # the latest response of model
    tool_call: Optional[ChatCompletionMessageToolCall] = (
        None  # the tool that the model requires
    )

In [125]:
model: OpenAI = OpenAI(temperature=0, model="gpt-4-1106-preview")
# model: Groq = Groq(
#     model="mixtral-8x7b-32768",
#     temperature=0,
#     timeout=60,
# )


# Convert python function to Llama-Index function and
# then OpenAI scheme.

func_tools: Sequence[BaseTool] = list(
    map(
        lambda f: FunctionTool.from_defaults(fn=f),
        [random_joke, translate],
    )
)
func_tools_dict: Dict[str, FunctionTool] = {
    tool.metadata.name: tool for tool in func_tools
}
openai_tools: Sequence[Dict[str, Any]] = list(
    map(lambda func_tool: func_tool.metadata.to_openai_tool(), func_tools)
)

In [140]:
def continue_next(
    state: GraphState,
) -> Literal["to_call_tool_func", "to_end"]:

    def _should_continue(chat_response: ChatResponse) -> bool:
        """To decide whether a tool function will be called (true) or not."""
        return (
            chat_response.message.additional_kwargs.get("tool_calls", None) is not None
        )

    if _should_continue(state["chat_response"]):
        state["tool_call"] = state["chat_response"].message.additional_kwargs[
            "tool_calls"
        ][0]
        return "to_run_tool"
    else:
        return "to_finish"


def run_agent(state: GraphState) -> Dict[str, Any]:
    chat_response: ChatResponse = model.chat(state["messages"], tools=openai_tools)
    ai_message: ChatMessage = chat_response.message

    pretty_print("AI messsaging", ai_message)

    state["messages"].extend([ai_message])
    return {"messages": state["messages"], "chat_response": chat_response}


def run_tool(state: GraphState) -> Dict[str, Sequence[ChatMessage]]:
    tool_call = state["tool_call"]

    func_id: str = tool_call.id
    func_name: str = tool_call.function.name
    args_json: str = tool_call.function.arguments

    func: FunctionTool = func_tools_dict.get(func_name)
    res: ToolOutput = func(**json.loads(args_json))

    pretty_print("Ran tool", res)

    func_message = ChatMessage(
        role=MessageRole.TOOL,
        content=str(res),
        name=func_name,
        additional_kwargs={
            "tool_call_id": func_id,
            "name": func_name,
        },
    )

    state["messages"].extend([func_message])
    return {"messages": state["messages"]}

In [141]:
workflow = StateGraph(GraphState)

workflow.add_node("run_agent", run_agent)
workflow.add_node("run_tool", run_tool)

workflow.set_entry_point("run_agent")
workflow.add_edge("run_tool", "run_agent")

workflow.add_conditional_edges(
    "run_agent", # start node name
    continue_next, # decision of what to do next AFTER start-node, the input is the output of the start-node
    {   # keys: return of continue_next, values: next node to continue
        "to_run_tool": "run_tool",
        "to_finish": END,
    },
)

app = workflow.compile()

In [142]:
human_input: str = """
Please create a random joke and provide the translated result in Simplified Chinese
without the original language, the final result should have no newlines, empty spaces, or instruction information text.
"""
def create_messages(human_input: str) -> Sequence[ChatMessage]:
    """Create a sequence of ChatMessage from a string input."""
    messages: Sequence[ChatMessage] = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=("You are an assisant to perform the user input."),
        ),
        ChatMessage(
            role=MessageRole.USER,
            content=("{input}"),
        ),
    ]
    prompt_template: ChatPromptTemplate = ChatPromptTemplate(messages)
    return prompt_template.format_messages(input=human_input)

result = app.invoke({"messages": create_messages(human_input=human_input)})
pretty_print("Result:", result)

AI messsaging


ChatMessage(
│   role=<MessageRole.ASSISTANT: 'assistant'>,
│   content=None,
│   additional_kwargs={
│   │   'tool_calls': [
│   │   │   ChatCompletionMessageToolCall(
│   │   │   │   id='call_BTWuX0GSm42BhIuiUdMoGUeH',
│   │   │   │   function=Function(arguments='{}', name='random_joke'),
│   │   │   │   type='function'
│   │   │   )
│   │   ]
│   }
)

Ran tool


ToolOutput(
│   content="What do you call a joke that's too long?\n\nA longshot!",
│   tool_name='random_joke',
│   raw_input={'args': (), 'kwargs': {}},
│   raw_output="What do you call a joke that's too long?\n\nA longshot!"
)

AI messsaging


...

ToolOutput(
│   content='Longshot。\n\nThis is a slang term for a joke that is too long.',
│   tool_name='translate',
│   raw_input={'args': (), 'kwargs': {'string': "What do you call a joke that's too long?\n\nA longshot!"}},
│   raw_output='Longshot。\n\nThis is a slang term for a joke that is too long.'
)

AI messsaging


ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='长篇大论。', additional_kwargs={})

Result:


{
│   'messages': [
│   │   ChatMessage(
│   │   │   role=<MessageRole.SYSTEM: 'system'>,
│   │   │   content='You are an assisant to perform the user input.',
│   │   │   additional_kwargs={}
│   │   ),
│   │   ChatMessage(
│   │   │   role=<MessageRole.USER: 'user'>,
│   │   │   content='\nPlease create a random joke and provide the translated result in Simplified Chinese\nwithout the original language, the final result should have no newlines, empty spaces, or instruction information text.\n',
│   │   │   additional_kwargs={}
│   │   ),
│   │   ChatMessage(
│   │   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   │   content=None,
│   │   │   additional_kwargs={
│   │   │   │   'tool_calls': [
│   │   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   │   id='call_BTWuX0GSm42BhIuiUdMoGUeH',
│   │   │   │   │   │   function=Function(arguments='{}', name='random_joke'),
│   │   │   │   │   │   type='function'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   }
│   │   ),
│   │   ChatMessage(
│   │   │   role=<MessageRole.TOOL: 'tool'>,
│   │   │   content="What do you call a joke that's too long?\n\nA longshot!",
│   │   │   additional_kwargs={'tool_call_id': 'call_BTWuX0GSm42BhIuiUdMoGUeH', 'name': 'random_joke'}
│   │   ),
│   │   ChatMessage(
│   │   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   │   content=None,
│   │   │   additional_kwargs={
│   │   │   │   'tool_calls': [
│   │   │   │   │   ChatCompletionMessageToolCall(
│   │   │   │   │   │   id='call_VrgXPeweC29ISzxyntac8RCX',
│   │   │   │   │   │   function=Function(
│   │   │   │   │   │   │   arguments='{"string":"What do you call a joke that\'s too long?\\n\\nA longshot!"}',
│   │   │   │   │   │   │   name='translate'
│   │   │   │   │   │   ),
│   │   │   │   │   │   type='function'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   }
│   │   ),
│   │   ChatMessage(
│   │   │   role=<MessageRole.TOOL: 'tool'>,
│   │   │   content='Longshot。\n\nThis is a slang term for a joke that is too long.',
│   │   │   additional_kwargs={'tool_call_id': 'call_VrgXPeweC29ISzxyntac8RCX', 'name': 'translate'}
│   │   ),
│   │   ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='长篇大论。', additional_kwargs={})
│   ],
│   'chat_response': ChatResponse(
│   │   message=ChatMessage(
│   │   │   role=<MessageRole.ASSISTANT: 'assistant'>,
│   │   │   content='长篇大论。',
│   │   │   additional_kwargs={}
│   │   ),
│   │   raw={
│   │   │   'id': 'chatcmpl-8zW95VDlrnf0r7niOLlFDn046debK',
│   │   │   'choices': [
│   │   │   │   Choice(
│   │   │   │   │   finish_reason='stop',
│   │   │   │   │   index=0,
│   │   │   │   │   logprobs=None,
│   │   │   │   │   message=ChatCompletionMessage(
│   │   │   │   │   │   content='长篇大论。',
│   │   │   │   │   │   role='assistant',
│   │   │   │   │   │   function_call=None,
│   │   │   │   │   │   tool_calls=None
│   │   │   │   │   )
│   │   │   │   )
│   │   │   ],
│   │   │   'created': 1709672267,
│   │   │   'model': 'gpt-4-1106-preview',
│   │   │   'object': 'chat.completion',
│   │   │   'system_fingerprint': 'fp_021f187978',
│   │   │   'usage': CompletionUsage(completion_tokens=7, prompt_tokens=223, total_tokens=230)
│   │   },
│   │   delta=None,
│   │   additional_kwargs={}
│   ),
│   'tool_call': None
}